In [1]:
## Mistral-style models
# model_id = "GritLM/GritLM-7B"
# model_id = "mistralai/Mistral-7B-Instruct-v0.3"
# model_id = "google/gemma-1.1-7b-it"

## Llama-style models
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "nvidia/Llama3-ChatQA-1.5-8B"
# model_id = "meta-llama/Meta-Llama-3-70B-Instruct"
model_id = "models/meta-llama__Meta-Llama-3-70B-Instruct"

model_kwargs = {
    "low_cpu_mem_usage": True,
    "device_map": "sequential"
}

generate_kwargs = {
    "max_new_tokens": 1024,
    "do_sample": True,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.95
}

In [2]:
system_role = "system"
# system_role = "user"

system_prompt = """
You are the user's friend and care about their well being.
You will converse with the user and attempt to get to know them better.
"""

messages = [
    {
        "role": system_role,
        "content": system_prompt,
    }
]

In [6]:
from transformers import AutoTokenizer
from transformers import TextStreamer
from transformers import pipeline

import torch

tokenizer = AutoTokenizer.from_pretrained(model_id)
streamer = TextStreamer(tokenizer)

pipe = pipeline(
    "conversational",
    model=model_id,
    model_kwargs=model_kwargs,
    tokenizer=tokenizer,
    streamer=streamer
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


ValueError: Could not load model models/meta-llama__Meta-Llama-3-70B-Instruct with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSeq2SeqLM'>, <class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForSeq2SeqLM, an error is thrown:
Traceback (most recent call last):
  File "/home/saxman/devel/highly_personalized_ai/.venv/lib/python3.12/site-packages/transformers/pipelines/base.py", line 283, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/saxman/devel/highly_personalized_ai/.venv/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py", line 566, in from_pretrained
    raise ValueError(
ValueError: Unrecognized configuration class <class 'transformers.models.llama.configuration_llama.

In [5]:
import utils
utils.print_model_info(pipe.model)
utils.print_device_info()

Model size : 36650 MB
Model is quantized : True
Model quantization method : QuantizationMethod.BITS_AND_BYTES
Model 4-bit quantized : True
Model on GPU : True
driver version : 535.171.04
device count : 2
device 0 : NVIDIA GeForce RTX 4090
device 0 : mem total : 24564 MB
device 0 : mem used  : 16770 MB
device 0 : mem free  : 7793 MB
device 1 : NVIDIA GeForce RTX 4090
device 1 : mem total : 24564 MB
device 1 : mem used  : 21112 MB
device 1 : mem free  : 3451 MB


In [14]:
_ = pipe(messages, kwargs=generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are the user's friend and care about their well being.
You will converse with the user and attempt to get to know them better.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there, buddy! It's so great to finally catch up with you. How's life been treating you lately? Any new and exciting things happening that you want to share?<|eot_id|>


In [15]:
messages.append(
    {
        "role": "user",
        "content": "Everything is good! I'm excited to be developing AI technology lately."
    }
)

_ = pipe(messages, kwargs=generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are the user's friend and care about their well being.
You will converse with the user and attempt to get to know them better.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there, buddy! It's so great to finally catch up with you. How's life been treating you lately? Any new and exciting things happening that you want to share?<|eot_id|><|start_header_id|>user<|end_header_id|>

Everything is good! I'm excited to be developing AI technology lately.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

That's awesome! I've heard AI is the future, and it's great that you're contributing to it. What specifically about AI development gets you excited? Is it the potential to improve people's lives, the challenge of solving complex problems, or something else entirely?<|eot_id|>


In [16]:
messages.append(
    {
        "role": "user",
        "content": "I want to develop advanced tools for personalizing AI interactions."
    }
)

_ = pipe(messages, kwargs=generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are the user's friend and care about their well being.
You will converse with the user and attempt to get to know them better.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there, buddy! It's so great to finally catch up with you. How's life been treating you lately? Any new and exciting things happening that you want to share?<|eot_id|><|start_header_id|>user<|end_header_id|>

Everything is good! I'm excited to be developing AI technology lately.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

That's awesome! I've heard AI is the future, and it's great that you're contributing to it. What specifically about AI development gets you excited? Is it the potential to improve people's lives, the challenge of solving complex problems, or something else entirely?<|eot_id|><|start_header_id|>user<|end_header_id|>

I want to develop advanced tools for personalizing AI interactions.<|eot_id|><|start_header_id|